In [1]:
import pandas as pd
import os
import torch.nn as nn
import pandas as pd
import datasets
from transformers import LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments, LongformerConfig, T5Config
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import wandb
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"



In [2]:
testing = True
run_cntr = 8

In [3]:
root = '/home/ugrads/a/aa_ron_su/physionet.org/files/clinical-t5/1.0.0/'
data_path = '/data/datasets/mimiciv_notes/physionet.org/files/mimic-iv-note/2.2/note/discharge.csv'
temivef_train_NOTE_TARGET1_FT_path = '/home/ugrads/a/aa_ron_su/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_train_NOTE_TARGET1_FT_rad.csv'
# model_name = 'Clinical-Longformer' # 'Clinical-T5-Base'
model_name = 'Clinical-T5-Base'
model_path = "yikuan8/Clinical-Longformer"
out_dir = f"{model_name}_{'test_' if testing else ''}out/{run_cntr}"

In [4]:
from transformers import T5Config, AutoConfig
from T5EncoderForSequenceClassification import T5EncoderForSequenceClassification
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForSequenceClassification, AutoModel

#clinLF:
# tokenizer = AutoTokenizer.from_pretrained(model_path)
# classifier = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                            # gradient_checkpointing = True)

#T5:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
encoder = model.get_encoder() # we only need the clinical-t5 encoder for our purposes

config_new = encoder.config
config_new.num_labels=2
config_new.last_hidden_size=64

classifier = T5EncoderForSequenceClassification(encoder, encoder.config)

Some weights of the model checkpoint at Clinical-T5-Base were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# label, input_ids, attention_mask = train_data[0:5].values()

In [6]:
# encoder.device

In [7]:
# encoder(    input_ids=input_ids,
#             attention_mask=attention_mask,
#             # inputs_embeds=inputs_embeds,
#             # head_mask=head_mask,
#             # output_attentions=output_attentions,
#             # output_hidden_states=output_hidden_states,
#             # return_dict=return_dict,
#             )

In [8]:
# encoder.config.save_pretrained(f"{out_dir}")

In [9]:
train = pd.read_csv(temivef_train_NOTE_TARGET1_FT_path)
print(f"reading notes and target from {temivef_train_NOTE_TARGET1_FT_path}")

reading notes and target from /home/ugrads/a/aa_ron_su/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_train_NOTE_TARGET1_FT_rad.csv


In [10]:
def group_train_test(ID):
    ID             = ID.astype(int)
    ID_unique_srtd = np.unique(ID)
    np.random.shuffle(ID_unique_srtd)    

    num_train_ids = int(.80 * len(ID_unique_srtd))
    train_ids = ID_unique_srtd[:num_train_ids]
    val_ids = ID_unique_srtd[num_train_ids:]

    train = ID[ID.isin(train_ids)]
    val = ID[ID.isin(val_ids)]

    assert(len(train) + len(val) == len(ID))
    assert(len(train_ids) + len(val_ids) == len(ID_unique_srtd))
    assert(len(train_ids) + len(val_ids) == len(ID_unique_srtd))

    return list(train.index), list(val.index)

train_idxs, val_idxs = group_train_test(train['ICUSTAY_ID'])

In [11]:
from datasets import Dataset
testing = True

target = 'delta_in_2_days'
train = train.rename(columns = {target:'label'})

train_data = train.iloc[train_idxs]
val_data = train.iloc[val_idxs]

if testing:
    train_data = train_data.iloc[:500]
    val_data = val_data.iloc[:500]

train_data = Dataset.from_pandas(train_data).select_columns(['text', 'label'])
val_data = Dataset.from_pandas(val_data).select_columns(['text', 'label'])

if not os.path.exists(f'{out_dir}/data_cache'):
    # define a function that will tokenize the model, and will return the relevant inputs for the model
    def tokenization(batched_text):
        return tokenizer(batched_text['text'], padding = 'max_length', truncation=True, max_length = 512)

    train_data = train_data.map(tokenization, batched = True, batch_size = len(train_data) // 10)
    val_data = val_data.map(tokenization, batched = True, batch_size = len(val_data) // 10)

    train_data.save_to_disk(f'{out_dir}/data_cache/tokenized_train_data')
    val_data.save_to_disk(f'{out_dir}/data_cache/tokenized_val_data')

else: 
    print(f'loading train, val from', f'{out_dir}/data_cache/')
    train_data = train_data.load_from_disk(f'{out_dir}/data_cache/tokenized_train_data')
    val_data = val_data.load_from_disk(f'{out_dir}/data_cache/tokenized_val_data')

train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

train_data = train_data.remove_columns('text')
val_data = val_data.remove_columns('text')

loading train, val from Clinical-T5-Base_test_out/8/data_cache/


In [12]:
# define accuracy metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # argmax(pred.predictions, axis=1)
    #pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [13]:
# define the training arguments
training_args = TrainingArguments(
    output_dir = f'{out_dir}/results',
    num_train_epochs = 1,
    per_device_train_batch_size = 3,
    gradient_accumulation_steps = 8,    
    per_device_eval_batch_size= 6,
    evaluation_strategy = "epoch",
    save_strategy="epoch",
    disable_tqdm = False, 
    load_best_model_at_end=True,
    warmup_steps=200,
    weight_decay=0.01,
    logging_steps = 8,
    fp16 = True,
    logging_dir=f'{out_dir}/logs',
    dataloader_num_workers = 0,
    run_name = 't5_radiology_run1',
)

In [14]:
# instantiate the trainer class and check for available devices
from MyTrainer import MyTrainer

trainer = MyTrainer(
    model=classifier,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
    # callbacks = [save_model_callback]
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

Using cuda_amp half precision backend


'cuda'

In [15]:
trainer.train()

encoder_outputs: BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]], device='cuda:2',
       grad_

***** Running Evaluation *****
  Num examples = 500
  Batch size = 18


encoder_outputs: BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ...

KeyboardInterrupt: 

In [18]:
trainer2 = Trainer(
    model=classifier,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
    # callbacks = [save_model_callback]
)

Using cuda_amp half precision backend


In [19]:
print(trainer2)

In [21]:
trainer2.train()

***** Running training *****
  Num examples = 500
  Num Epochs = 1
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 72
  Gradient Accumulation steps = 8
  Total optimization steps = 7
  Number of trainable parameters = 110258498


encoder_outputs: BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]], device='cuda:0',
       grad_

Epoch,Training Loss,Validation Loss


encoder_outputs: BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]], device='cuda:0',
       grad_

KeyboardInterrupt: 

In [ ]:
train_data['input_ids'].size(), train_data['label'].size()

(torch.Size([500, 512]), torch.Size([500]))

In [ ]:
train_data

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 500
})

In [ ]:
training_args

In [ ]:
os.environ["WANDB_DISABLED"] = 'true'

In [15]:
# import wandb
# wandb.init()
# print(wandb.run.get_url())
trainer.train()


/home/ugrads/a/aa_ron_su/miniconda3/envs/clinical1/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 500
  Num Epochs = 1
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 72
  Gradient Accumulation steps = 8
  Total optimization steps = 7
  Number of trainable parameters = 110258498
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aa_ron_su (aaron_team). Use `wandb login --relogin` to force relogin


encoder_outputs: BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]], device='cuda:0',
       grad_

/home/ugrads/a/aa_ron_su/miniconda3/envs/clinical1/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


encoder_outputs: BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]], device='cuda:1',
       grad_

Epoch,Training Loss,Validation Loss


encoder_outputs: BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]], device='cuda:2',
       grad_

KeyboardInterrupt: 

In [ ]:
# from transformers import DefaultFlowCallback

# class SaveModelCallback(TrainerCallback):
#     def __init__(self, output_dir):
#         self.output_dir = output_dir

#     def on_epoch_end(self, args, state, control, **kwargs):
#         model = state.model
#         epoch = state.epoch
#         ckpt_dir = os.path.join(self.output_dir, f"epoch_{epoch}")
#         os.makedirs(ckpt_dir, exist_ok=True)
#         model.save_pretrained(ckpt_dir)

# save_model_callback = SaveModelCallback(output_dir=out_dir)

In [ ]:
# best_model = trainer._load_best_model() # doesn't work

In [ ]:
# torch.save(best_model, f'{out_dir}/best_model.pt')

In [ ]:
# from torch.utils.data import DataLoader, TensorDataset, RandomSampler

# train_inputs = train_tensor
# train_labels = torch.tensor(train['delta_in_2_days'].to_numpy())
# train_dataset = TensorDataset(train_inputs.to(device), train_labels.to(device)) 

In [ ]:
# from torch.utils.data import Subset
# batch_size = 4


# for fold, (train_idx, val_idx) in enumerate(kfold):
#     print(train_idx.shape, val_idx.shape)
#     val_set = Subset(train_dataset, val_idx)
#     val_dataloader = DataLoader(val_set, batch_size=batch_size, shuffle=True)
#     train_set = Subset(train_dataset, train_idx)
#     train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
#     print(len(data_loader))

In [ ]:
# notes_to_extract = df['text']
# texts = notes_to_extract.tolist()
# tokenized_notes_to_extract = tokenizer(texts, truncation=True, padding=True, return_tensors = "pt")

# # test_note_to_extract = notes_to_extract.iloc[0]
# # tokenized_test_note = tokenize_function(test_note_to_extract)
# # tokenized_test_note.keys()

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
# from T5EncoderForSequenceClassification import T5EncoderForSequenceClassification, T5EncoderClassificationHead

In [ ]:
# classifier.named_parameters

In [ ]:
# import torch
# import torch.nn as nn
# from torch.nn.utils import parameters_to_vector

# num_params = len(parameters_to_vector(encoder.parameters()))

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device

In [ ]:
# classifier.to(device)

In [ ]:
# num_params

In [ ]:
# len(parameters_to_vector(classifier.classifier.parameters()))

In [ ]:
# classifier.encoder.named_parameters

In [ ]:
# classifier.classifier

In [ ]:
# from torch.utils.data import DataLoader, TensorDataset, RandomSampler
# from transformers import AdamW, get_linear_schedule_with_warmup
# import numpy as np
# import torch

# # Define your training data
# train_inputs = tokenized_notes_to_extract.input_ids
# # train_labels = torch.tensor(np.random.rand(len(train_inputs)))
# train_labels = torch.tensor(df['delta_in_2_days'].to_numpy())
# train_dataset = TensorDataset(train_inputs.to(device), train_labels.to(device))

In [ ]:
# train_dataset[0][0].device.type

In [ ]:
# batch_size = 32
# train_sampler = RandomSampler(train_dataset)
# train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

In [ ]:
# batch_size = 16
# num_epochs = 1
# learning_rate = 5e-5
# adam_epsilon = 1e-8
# max_grad_norm = 1.0

# train_dataloader = DataLoader(train_dataset, batch_size=batch_size)

In [ ]:
# import torch.optim as optim

# criterion = nn.MSELoss()
# optimizer = optim.AdamW(classifier.classifier.parameters(), lr=learning_rate, eps=adam_epsilon)

In [ ]:
# total_steps = len(train_dataloader) * num_epochs
# scheduler = get_linear_schedule_with_warmup(
#     optimizer,
#     num_warmup_steps=0,
#     num_training_steps=total_steps
# )

In [ ]:
# # freeze encoder weights:
# for param in classifier.encoder.parameters():
#     param.requires_grad = False

In [ ]:
# from tqdm import tqdm

# classifier.classifier.train()
# for epoch in tqdm(range(num_epochs)):
#     for step, batch in enumerate(train_dataloader):
#         inputs, labels = batch
#         optimizer.zero_grad()
#         outputs = classifier.forward(inputs, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(classifier.classifier.parameters(), max_grad_norm)
#         optimizer.step()
#         scheduler.step()

#         # Print progress every 10 steps
#         if step % 10 == 0:
#             print(f"Epoch {epoch + 1}/{num_epochs} | Step {step}/{len(train_dataloader)} | Loss {loss.item():.4f}")

### EXTRACT NOTES

In [ ]:
# single_embedding = classifier.forward(train_inputs[0:2], labels = train_labels[0:2], return_embeddings=True)

In [ ]:
# embeddings = classifier.forward(train_inputs, labels = train_labels, return_embeddings=True)
# embeddings_df = pd.DataFrame({'embedding': list(embeddings.detach().numpy())})
# df.reset_index(drop=True, inplace=True)
# df = pd.concat([df, embeddings_df], axis = 1)
# df.to_csv(mimic_iv_train_NOTE_EMBEDDINGS_path, index = False)

In [ ]:
# embeddings_df = pd.DataFrame({'embedding': list(embeddings.detach().numpy())})

In [ ]:
# df_small = df.iloc[0:5].copy().reset_index(drop=True)

In [ ]:
# df_small = pd.concat([df_small, embeddings_df], axis = 1)

In [ ]:
# Merge embeddings with dataframe

In [ ]:
# embeddings[0].shape

In [ ]:
# df_small

In [ ]:
# 

In [ ]:
# outputs['classifier_last_hidden_state'][0].shape
# outputs['logits'][0]


In [ ]:
# encoder_with_dense = T5EncoderWithDense(encoder = encoder, num_classes = 1)

In [ ]:
# dense_layer_outputs = encoder_with_dense(inputs.input_ids)

In [ ]:
# encoder_with_dense.hidden_states

In [ ]:
# dense_layer_outputs

In [ ]:
# dense(pooled_output)

In [ ]:
# # Freeze the weights of the encoder layers
# for param in model_encoder_only.parameters():
#     param.requires_grad = False

In [ ]:
# from datasets import load_dataset
# # dataset = load_dataset('csv', data_files=mimic_iv_train_NOTE_path, split='train') # split = 'train
# df = pd.read_csv(mimic_iv_train_NOTE_path)[['NOTE_ID', 'text']]
# df.drop_duplicates(inplace=True)
# df.dropna(inplace=True) 
# df

In [ ]:
# import torch
# inputs = tokenized_test_note
# outputs = tokenized_test_note
# outputs = model(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask, decoder_input_ids = inputs.input_ids)

In [ ]:
# outputs.keys()

In [ ]:
# # import torch
# inputs = tokenized_test_note
# labels = torch.tensor([1]).unsqueeze(0)
# outputs = model_encoder_only

In [ ]:
# generated_outputs = []
# labels = torch.tensor([1]).unsqueeze(0)

# for i, row in notes_to_extract.iterrows():
#     input_text = row['text']
#     input_ids = tokenizer.encode(input_text, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
#     outputs = model(**inputs, labels=labels)
#     generated_outputs.append(outputs)

In [ ]:
# import numpy as np

# # Generate a random target dataset with n = 1000
# n = 1000
# target_df = np.random.rand(n)